In [1]:
# load instructed-tuned ckpt
from inference_module import SFMGenerator
from tqdm import tqdm
from collections import defaultdict

ckpt_home = '/home/yinxia/blob1.v2/yinxia/scigpt/7bv3/overall_instruct_20240430/run2/global_step24128'
generator = SFMGenerator(ckpt_home)

/home/yinxia/miniconda3/envs/sfm-scigptv3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-24 01:31:56,629] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'SFMDecTokenizer'.
You are using the default legacy behaviour of the <class 'sfm.data.sci_data.SFMDecTokenizer.SFMDecTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


[2024-05-24 01:31:59.042][INFO]: Loading protein sentencepiece model from /home/yinxia/blob1.v2/shufxi/data/scigpt/ur50bpe/bpe.model and /home/yinxia/blob1.v2/shufxi/data/scigpt/ur50bpe/bpe.vocab
[2024-05-24 01:31:59.867][INFO]: Loading DNA sentencepiece model from /home/yinxia/blob1.v2/shufxi/data/scigpt/dnabpe/bpe.model and /home/yinxia/blob1.v2/shufxi/data/scigpt/dnabpe/bpe.vocab
[2024-05-24 01:32:00.074][INFO]: Loading RNA sentencepiece model from /home/yinxia/blob1.v2/shufxi/data/scigpt/rnabpe/bpe.model and /home/yinxia/blob1.v2/shufxi/data/scigpt/rnabpe/bpe.vocab
[2024-05-24 01:32:00.518][INFO]: Tokenizer has 40014 tokens


Loading checkpoint shards: 100%|██████████| 2/2 [00:54<00:00, 27.43s/it]


layer0 embed_tokens.weight torch.Size([40014, 4096])
01 self_attn.q_proj.weight torch.Size([4096, 4096])
01 self_attn.k_proj.weight torch.Size([4096, 4096])
01 self_attn.v_proj.weight torch.Size([4096, 4096])
01 self_attn.o_proj.weight torch.Size([4096, 4096])
01 mlp.gate_proj.weight torch.Size([11008, 4096])
01 mlp.up_proj.weight torch.Size([11008, 4096])
01 mlp.down_proj.weight torch.Size([4096, 11008])
01 input_layernorm.weight torch.Size([4096])
01 post_attention_layernorm.weight torch.Size([4096])
02 self_attn.q_proj.weight torch.Size([4096, 4096])
02 self_attn.k_proj.weight torch.Size([4096, 4096])
02 self_attn.v_proj.weight torch.Size([4096, 4096])
02 self_attn.o_proj.weight torch.Size([4096, 4096])
02 mlp.gate_proj.weight torch.Size([11008, 4096])
02 mlp.up_proj.weight torch.Size([11008, 4096])
02 mlp.down_proj.weight torch.Size([4096, 11008])
02 input_layernorm.weight torch.Size([4096])
02 post_attention_layernorm.weight torch.Size([4096])
03 self_attn.q_proj.weight torch.Size

In [2]:
from collections import defaultdict
from evaluate_text2material import parse_material_string, smact_validity

query = 'Develop a substance that features a band gap of 0 eV '

ret_mat = {}
fail, total = 0, 0
# beam search outputs
for _ in range(10):
    ret = generator.chat(query, do_sample=True, max_new_tokens=200)
    for r in ret:
        material = parse_material_string(r)
        if material is None:
            fail += 1
            continue
        comp, count = [], []
        for k,v in material[0].items():
            comp.append(k)
            count.append(v)
        
        if smact_validity(tuple(comp), tuple(count),use_element_symbol=True):
            k = tuple([tuple(comp), tuple(count)])
            v = material[1]
            if k not in ret_mat:
                ret_mat[k] = {}
            if v not in ret_mat[k]:
                ret_mat[k][v] = 0
            ret_mat[k][v] += 1

In [3]:
ret_mat

{(('Ti', 'C'), (1, 2)): {'<sg166>': 1},
 (('Gd', 'Al', 'Pd'), (2, 4, 2)): {'<sg63>': 1},
 (('Yb', 'Mg', 'Tl'), (1, 1, 2)): {'<sg160>': 1},
 (('La', 'Ge', 'Ir'), (4, 8, 4)): {'<sg71>': 1},
 (('Cs', 'Mg'), (1, 15)): {'<sg187>': 1},
 (('Li', 'Al', 'Ru'), (1, 2, 1)): {'<sg71>': 1},
 (('Nb', 'Re', 'Mo'), (1, 1, 2)): {'<sg71>': 1},
 (('Yb', 'Sm', 'Rh'), (1, 1, 2)): {'<sg160>': 1},
 (('Mg', 'Al', 'Ag'), (17, 11, 1)): {'<sg8>': 1},
 (('K', 'As', 'O'), (1, 1, 3)): {'<sg221>': 1},
 (('Yb', 'Cu', 'Sn'), (2, 2, 2)): {'<sg194>': 1},
 (('La', 'Cd', 'Ir'), (16, 4, 4)): {'<sg1>': 1},
 (('K', 'Ir'), (3, 1)): {'<sg139>': 1},
 (('Yb', 'Al', 'Zn'), (1, 1, 2)): {'<sg160>': 1},
 (('V', 'Ga'), (12, 14)): {'<sg42>': 1},
 (('Ba', 'Mg', 'Tl'), (2, 1, 1)): {'<sg71>': 1},
 (('Zr', 'Al', 'Co'), (6, 2, 1)): {'<sg6>': 1},
 (('Dy', 'Cu', 'S'), (1, 2, 2)): {'<sg164>': 1},
 (('Y', 'Ge'), (22, 20)): {'<sg1>': 1},
 (('Pr', 'Al', 'Cu'), (3, 3, 3)): {'<sg38>': 1},
 (('Sc', 'Tl', 'Rh'), (2, 1, 1)): {'<sg71>': 1},
 (('Mg', '

In [4]:
query = 'Generate a material containing Si, Al and other elements'

ret_mat = {}
fail, total = 0, 0

# to use beam search outputs, set `do_sample=False`
# to use rand search outputs, set `do_sample=True`

for _ in range(2):
    ret = generator.chat(query, do_sample=True, max_new_tokens=200)
    for r in ret:
        material = parse_material_string(r)
        if material is None:
            fail += 1
            continue
        comp, count = [], []
        for k,v in material[0].items():
            comp.append(k)
            count.append(v)
        
        if smact_validity(tuple(comp), tuple(count),use_element_symbol=True):
            k = tuple([tuple(comp), tuple(count)])
            v = material[1]
            if k not in ret_mat:
                ret_mat[k] = {}
            if v not in ret_mat[k]:
                ret_mat[k][v] = 0
            ret_mat[k][v] += 1

In [5]:
ret_mat

{(('Mg', 'Al', 'Si', 'O'), (6, 1, 1, 8)): {'<sg123>': 1},
 (('Ca', 'Al', 'Sb'), (40, 2, 36)): {'<sg9>': 1}}

In [9]:
query = 'Illustrate a material with zero band gap, containing Si, Al and other elements'

ret_mat = {}
fail, total = 0, 0

for _ in range(2):
    ret = generator.chat(query, do_sample=True, max_new_tokens=200)
    for r in ret:
        material = parse_material_string(r)
        if material is None:
            fail += 1
            continue
        comp, count = [], []
        for k,v in material[0].items():
            comp.append(k)
            count.append(v)
        
        if smact_validity(tuple(comp), tuple(count),use_element_symbol=True):
            k = tuple([tuple(comp), tuple(count)])
            v = material[1]
            if k not in ret_mat:
                ret_mat[k] = {}
            if v not in ret_mat[k]:
                ret_mat[k][v] = 0
            ret_mat[k][v] += 1

In [10]:
ret_mat

{(('Ce', 'Al', 'Si', 'Ni'), (2, 11, 13, 56)): {'<sg1>': 1},
 (('Mg', 'Al', 'Si', 'O'), (30, 1, 1, 32)): {'<sg123>': 2}}